Apply the epidemiology classification model to PubMed searches for several rare diseases

In [1]:
import pandas as pd
import classify_abs
import requests
import xml.etree.ElementTree as ET
import spacy

In [2]:
pd.set_option('display.max_colwidth', -1)

/home/jennifernj/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [3]:
nlp = spacy.load('en_core_web_lg')
nlpSci = spacy.load("en_ner_bc5cdr_md")
nlpSci2 = spacy.load('en_ner_bionlp13cg_md')

In [4]:
def rank_results(dz, maxResults):
    dz_results = pd.DataFrame(columns=['pmid', 'abstract', 'epi_prob', 'is_epi'])

    # get results from searching for disease name through EBI API
    term = ''
    dz_words = dz.split()
    for word in dz_words:
        term += word + '%20'
    query = term[:-3]
    url = 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query='+query+'&resulttype=core'
    r = requests.get(url)
    root = ET.fromstring(r.content)

    pmid_to_abs = {}
    i = 0

    # loop over resulting articles
    for result in root.iter('result'):
        if i >= maxResults:
            break
        pmids = [pmid.text for pmid in result.iter('id')]
        if len(pmids) > 0:
            pmid = pmids[0]
            if pmid[0].isdigit():
                abstracts = [abstract.text for abstract in result.iter('abstractText')]
                if len(abstracts) > 0:
                    pmid_to_abs[pmid] = abstracts[0]
                    i += 1

    # predict on each PubMed ID & add to dataframe
    for pmid in pmid_to_abs:
        prob, isEpi = classify_abs.getPredictions(pmid, nlp, nlpSci, nlpSci2, 'my_model_orphanet_final')
        dz_results = dz_results.append({'pmid':pmid, 'abstract':pmid_to_abs[pmid], 'epi_prob':prob
                                         , 'is_epi':isEpi}, ignore_index=True)
    return dz_results.sort_values('epi_prob', ascending=False)
    

In [11]:
tay_sachs = rank_results('tay sachs', 50)

In [7]:
turner_syndrome = rank_results('turner syndrome', 50)

In [8]:
sickle_cell = rank_results('sickle cell', 50)

In [9]:
cystic_fibrosis = rank_results('cystic fibrosis', 50)

In [10]:
ehlers_danlos = rank_results('ehlers danlos', 50)

In [14]:
tay_sachs.to_csv('proof_of_concept/tay_sachs.csv',index=False)
turner_syndrome.to_csv('proof_of_concept/turner_syndrome.csv',index=False)
sickle_cell.to_csv('proof_of_concept/sickle_cell.csv',index=False)
cystic_fibrosis.to_csv('proof_of_concept/cystic_fibrosis.csv',index=False)
ehlers_danlos.to_csv('proof_of_concept/ehlers_danlos.csv',index=False)

In [4]:
tay_sachs = pd.read_csv('proof_of_concept/tay_sachs.csv')

Get titles for Tay Sachs (for paper figure)

In [15]:
titles = []
for i,row in tay_sachs.iterrows():
    pmid = row['pmid']
    url = 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=EXT_ID:'+str(pmid)+'&resulttype=core'
    r = requests.get(url)
    root = ET.fromstring(r.content)
    for child in root.iter('*'):
        if child.tag == 'title':
            titles.append(child.text)
            break
print(len(titles), i)
tay_sachs['titles'] = titles

19 18


In [18]:
tay_sachs_figure = tay_sachs.iloc[:5][['titles','epi_prob']]

In [25]:
tay_sachs_figure.rename(columns={'epi_prob':'Probability of epidemiology'}, inplace=True)

In [31]:
tay_sachs_figure = tay_sachs_figure.round(3)

In [34]:
tay_sachs_figure['Relevant text'] = ['This corresponds to an incidence of 1/218,144, which in turn corresponds to a carrier frequency of 1/234.'
                                    ,'The disease is very rare in Turkey, with an incidence of 0.54/100,000'
                                    ,'TSD is more prevalent among Ashkenazi Jewish (AJ) individuals and some other genetically isolated populations with carrier frequencies of approximately ~1:27 which is much higher than that of 1:300 in the general population'
                                    ,'None'
                                    ,'(a calculated birth prevalence of 1 per 325,175 live births)']

In [35]:
tay_sachs_figure

,Title,Probability of epidemiology,Relevant text
0,"The incidence and carrier frequency of Tay-Sachs disease in the French-Canadian population of Quebec based on retrospective data from 24 years, 1992-2015.",0.999,"This corresponds to an incidence of 1/218,144, which in turn corresponds to a carrier frequency of 1/234."
1,Presentation of central precocious puberty in two patients with Tay-Sachs disease.,0.998,"The disease is very rare in Turkey, with an incidence of 0.54/100,000"
2,Prenatal Diagnosis of Tay-Sachs Disease.,0.823,TSD is more prevalent among Ashkenazi Jewish (AJ) individuals and some other genetically isolated populations with carrier frequencies of approximately ~1:27 which is much higher than that of 1:300 in the general population
3,Patient-Derived Phenotypic High-Throughput Assay to Identify Small Molecules Restoring Lysosomal Function in Tay-Sachs Disease.,0.644,None
4,"Amyotrophy, cerebellar impairment and psychiatric disease are the main symptoms in a cohort of 14 Czech patients with the late-onset form of Tay-Sachs disease.",0.622,"(a calculated birth prevalence of 1 per 325,175 live births)"


In [33]:
tay_sachs

,pmid,abstract,epi_prob,is_epi,titles
0,32302469,"Tay-Sachs disease (TSD) is a hereditary neurodegenerative condition inherited through an autosomal recessive pattern. The incidence and carrier frequency of infantile TSD were found to be increased among French Canadians in specific areas of the province of Quebec or calculated from New England populations with French-Canadian heritage. No accurate infantile TSD carrier frequency for the whole French-Canadian population in Quebec has been published. In this study, we estimate the incidence and carrier frequency of infantile TSD in the Quebec French-Canadian population. The number of TSD cases was ascertained during the 1992-2015 period, as well as the number of births to mothers whose language of use is French. Seven cases of TSD have been diagnosed in Quebec during the period of ascertainment. This corresponds to an incidence of 1/218,144, which in turn corresponds to a carrier frequency of 1/234. In the same 24-year period, there are two French-Canadian couples who had a fetus prenatally diagnosed with TSD. If these cases are included, the incidence of TSD in the French-Canadian population of Quebec is 1/169,668 and the carrier frequency 1/206. These findings can be used for genetic counseling and policy decisions regarding carrier screening for TSD in populations of French-Canadian descent.",0.998829,True,"The incidence and carrier frequency of Tay-Sachs disease in the French-Canadian population of Quebec based on retrospective data from 24 years, 1992-2015."
1,29943104,"Tay-Sachs disease is an autosomal recessive type of lysosomal storage disorder. The disease is very rare in Turkey, with an incidence of 0.54/100,000. The clinical manifestations of Tay-Sachs disease include progressive developmental delay, seizures, deafness, blindness, spasticity, and dystonia, which are caused by the accumulation of gangliosides in the central nervous system. To date, only one case indicating the association between Tay-Sachs disease and central precocious puberty has been reported. Although the mechanism of this association is not clear, it is thought to be due to ganglioside accumulation in the central nervous system or the inhibition of the hypothalamic inhibiting pathway. Herein, we report two patients with genetically proven Tay-Sachs disease who developed central precocious puberty during follow-up. Pubertal development in patients affected by Tay-Sachs disease should be carefully assessed.",0.998114,True,Presentation of central precocious puberty in two patients with Tay-Sachs disease.
2,30506202,"Tay-Sachs disease (TSD) is an autosomal recessive lysosomal storage disorder caused by mutations of the HEXA gene resulting in the deficiency of hexosaminidase A (Hex A) and subsequent neuronal accumulation of GM2 gangliosides. Infantile TSD is a devastating and fetal neurodegenerative disease with death before the age of 3-5 years. A small proportion of TSD patients carry milder mutations and may present juvenile or adult onset milder disease. TSD is more prevalent among Ashkenazi Jewish (AJ) individuals and some other genetically isolated populations with carrier frequencies of approximately ~1:27 which is much higher than that of 1:300 in the general population. Carrier screening and prenatal testing for TSD are effective in preventing the birth of affected fetuses greatly diminishing the incidence of TSD. Testing of targeted HEXA mutations by genotyping or sequencing can detect 98% of carriers in AJ individuals; however, the detection rate is much lower for most other ethnic groups. When combined with enzyme analysis, above 98% of carriers can be reliably identified regardless of ethnic background. Multiplex PCR followed by allele-specific primer extension is one method to test for known and common mutations. Sanger sequencing or other sequencing methods are useful to identify private mutations. For prenatal testing, only predefined parental mutations need to be tested. In the event o

In [16]:
turner_syndrome

,pmid,abstract,epi_prob,is_epi
19,32271829,"BACKGROUND:Turner syndrome (TS) is a chromosomal disorder, in which a female is partially or entirely missing one of the two X chromosomes, with a prevalence of 1:2500 live female births. The present study aims to identify a circulating microRNA (miRNA) signature for TS patients with and without congenital heart disease (CHD). METHODS:Microarray platform interrogating 2549 miRNAs were used to detect the miRNA abundance levels in the blood of 33 TS patients and 14 age-matched healthy volunteer controls (HVs). The differentially abundant miRNAs between the two groups were further validated by RT-qPCR. RESULTS:We identified 60 differentially abundant miRNA in the blood of TS patients compared to HVs, from which, 41 and 19 miRNAs showed a higher and a lower abundance levels in TS patients compared to HVs, respectively. RT-qPCR confirmed the significantly higher abundance levels of eight miRNAs namely miR-374b-5p, miR-199a-5p, miR-340-3p, miR-125b-5p, miR-30e-3p, miR-126-3p, miR-5695, and miR-26b-5p in TS patients as compared with the HVs. The abundance level of miR-5695 was higher in TS patients displaying CHD as compared to TS patients without CHD (p = 0.0265; log2-fold change 1.99); whereas, the abundance level of miR-126-3p was lower in TS patients with congenital aortic valve disease (AVD) compared to TS patients without BAV (p = 0.0139, log2-fold change 1.52). The clinical feature statistics revealed that miR-126-3p had a significant correlation with sinotubular junction Z-score (r = 0.42; p = 0.0154). CONCLUSION:The identified circulating miRNAs signature for TS patients with manifestations associated with cardiovascular diseases provide new insights into the molecular mechanism of TS that may guide the development of novel diagnostic approaches.",0.989621,True
8,32613319,"Turner syndrome (TS) is associated with coronary artery disease (CAD), an important cause of premature death in TS. However, the determinants of CAD in women with TS remain unknown. In a cross-sectional study design, 168 women without clinical evidence of CAD (115 with TS and 53 without TS) were assessed for the presence and volume of subclinical CAD using coronary CT angiography. Karyotype, the presence of congenital heart defects and conventional cardiovascular risk factors were also registered. Comparative analyses were performed (1) between women with and without TS and (2) in the TS group, between women with and without subclinical CAD. The prevalence of CAD, in crude and adjusted analyses, was not increased for women with TS (crude prevalence: 40 [35%] in TS vs. 25 [47%] in controls, p = 0.12). The volume of atherosclerosis was not higher in women with TS compared with controls (median and interquartile range 0 [0-92] in TS vs. 0 [0-81]mm3 in controls, p = 0.29). Among women with TS, women with subclinical CAD were older (46 ± 13 vs. 37 ± 11 years, p < 0.001), had higher blood pressure (systolic blood pressure 129 ± 16 vs. 121 ± 16 mmHg, p < 0.05) and were more frequently diagnosed with type 2 diabetes (5 [13%] vs. 2 [3%], p < 0.05). Karyotype or congenital heart defects were not associated with subclinical CAD. Some women with TS show early signs of CAD, however overall, not more than women without TS. Conventional cardiovascular risk factors were the principal determinants of CAD also in TS, and CAD prevention strategies should be observed.ClinicalTrial.gov Identifier: NCT01678261 ( https://clinicaltrials.gov/ct2/show/NCT01678261 ).",0.682994,True
12,32617534,"Background: The cognitive profile of Turner syndrome, a genetic disorder resulting from partial or complete X-chromosome deletion, presents characteristic deficits. Despite this, studies have yet to evaluate how deficits translate into and are compensated for in academic settings. This study seeks to explore cognitive functioning, as well as the accessibility and development of academic accommodations in females with Turner syndrome from adolescence

In [17]:
sickle_cell

pmid  \
3   32520675   
10  32387142   
4   32702096   
7   32674605   
1   32548653   
16  32462802   
12  32614473   
14  32394268   
0   32070266   
2   32525359   
13  32362160   
6   32722421   
11  32572524   
9   32673671   
8   32697889   
5   32564896   
15  32541442   
17  31983291   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [18]:
cystic_fibrosis

pmid  \
1   32540174   
5   32097175   
7   32670584   
15  32505496   
10  32718005   
13  32668165   
8   32475007   
3   32397175   
4   32358807   
2   32374578   
12  32678926   
0   32608531   
6   32497260   
14  32671834   
9   32703846   
11  32717740   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [19]:
ehlers_danlos

,pmid,abstract,epi_prob,is_epi
6,32518415,"PURPOSE:Cardiac-valvular and vascular Ehlers-Danlos syndrome (EDS) have significant cardiovascular issues. The prevalence and significance of such abnormalities in classical (cEDS) or hypermobile EDS (hEDS) remain unclear. We report the prevalence of cardiac abnormalities in patients with cEDS and hEDS. METHODS:We identified 532 pediatric patients with potential EDS evaluated at our institution from January 2014 through April 2019 by retrospective chart review. Ninety-five patients (12 cEDS and 83 hEDS patients) met 2017 EDS diagnostic criteria and had an echocardiogram. One patient was excluded due to complex congenital heart disease, and two were excluded due to lack of images. We reviewed echocardiograms for all structural abnormalities. RESULTS:Of these 95 patients, 1 had mild aortic root dilation, and 1 had mild ascending aorta dilation in the setting of a bicuspid aortic valve. Eleven patients (11.6%) had a cardiac valve abnormality, all of which were trivial to mild. None of the patients required cardiac intervention. CONCLUSION:Our results demonstrate that aortic dilation and valvular anomalies are uncommon in cEDS or hEDS patients. Given the lack of evidence, we do not recommend echocardiographic evaluation and surveillance in patients with cEDS and hEDS in the absence of clinical findings or positive family history.",0.997316,True
13,31947929,"Ehlers-Danlos Syndromes (EDS) are a group of heritable disorders of connective tissue (HDCT) characterized by joint hypermobility, skin hyperextensibility, and tissue fragility. Orthostatic intolerance (OI) is highly prevalent in EDS however mechanisms linking OI to EDS remain poorly understood. We hypothesize that impaired blood pressure (BP) and heart rate control is associated with lower arterial stiffness in people with EDS. Orthostatic vital signs and arterial stiffness were assessed in a cohort of 60 people with EDS (49 female, 36 ± 16 years). Arterial elasticity was assessed by central and peripheral pulse wave velocity (PWV). Central PWV was lower in people with EDS compared to reference values in healthy subjects. In participants with EDS, central PWV was correlated to supine systolic BP (r = 0.387, p = 0.002), supine diastolic BP (r = 0.400, p = 0.002), and seated systolic BP (r = 0.399, p = 0.002). There were no significant correlations between PWV and changes in BP or heart rate with standing (p > 0.05). Between EDS types, there were no differences in supine hemodynamics or PWV measures (p > 0.05). These data demonstrate that increased arterial elasticity is associated with lower BP in people with EDS which may contribute to orthostatic symptoms and potentially provides a quantitative clinical measure for future genotype-phenotype investigations.",0.993142,True
0,32654548,"PURPOSE:Ehlers-Danlos Syndrome (EDS) is an inherited connective tissue disorder caused by abnormal collagen synthesis. Little is known about its effects on pregnancy. The purpose of this study was to evaluate the pregnancy outcomes in women with EDS. MATERIALS AND METHODS:We conducted a population-based retrospective cohort study using the Healthcare Cost and Utilization Project Nationwide Inpatient Sample database from the United States. The study included women who delivered between 1999 and 2014. We measured the prevalence of EDS over time, and compared the baseline, obstetrical, and neonatal outcomes among women with EDS to the general obstetrical population without EDS. Unconditional logistic regression models were used to calculate the adjusted effect of EDS on maternal and neonatal outcomes. RESULTS:The overall prevalence of EDS in pregnancy was 7 per 100,000 births, with the trend increasing over the 16 year study period (p < .0001). Women with EDS were more likely to be Caucasian, belong to a higher income quartile, and smoke. Pregnancies in women with EDS were associated with prematurity, 1.47 (1.18-1.82), cervical incompetence, 3.11 (1.99-4.85)